In [66]:
import cv2
import numpy as np

def showImg(imgName, img):
    cv2.imshow(imgName, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [67]:
roadImg = cv2.imread('road2.jpg')
grayImg = cv2.cvtColor(roadImg,cv2.COLOR_BGR2GRAY)
blurImg = cv2.GaussianBlur(grayImg,(5,5),0)
edgesImg =  cv2.Canny(blurImg, 50, 150)
showImg('default',roadImg)
showImg('gray',grayImg)
showImg('blur',blurImg)
showImg('edges',edgesImg)
    

In [68]:
def maskFrame(image):
    grayImg = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    blurImg = cv2.GaussianBlur(grayImg,(5,5),0)
    edgesImg =  cv2.Canny(blurImg, 50, 150)
    
    verticles = np.array([
    [(0,image.shape[0]),
    (450,310),
    (490,310),
    (image.shape[1],image.shape[0])]
    ], dtype=np.int32)

    mask = np.zeros_like(edgesImg)
    cv2.fillPoly(mask,verticles,255)
    maskedEdges = cv2.bitwise_and(edgesImg, mask)
    return maskedEdges

In [69]:
maskedImg = maskFrame(roadImg)
showImg('masked',maskedImg)

In [70]:
def drawLines(img, lines, color= [0,0,255],thickness = 5):
    xBottomPos = []
    xUpperPos = []
    xBottomNeg = []
    xUpperNeg = []

    yBottom = 540
    yUpper = 315
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            val = (y2-y1)/(x2-x1)
            slope = val
            b = y1-slope*x1
            if val > 0.5 and val < 0.8:
                xBottomPos .append((yBottom - b)/slope)
                xUpperPos.append((yUpper-b)/slope)
            elif val <-0.5 and val > -0.8:
                xBottomNeg.append((yBottom - b)/slope)
                xUpperNeg.append((yUpper-b)/slope)
    if len(xBottomPos) > 0 and len(xBottomNeg) > 0:
        linesMean = np.array(
            [
                [int(np.mean(xBottomPos)), int(np.mean(yBottom)), int(np.mean(xUpperPos)), int(np.mean(yUpper))],
                [int(np.mean(xBottomNeg)), int(np.mean(yBottom)), int(np.mean(xUpperNeg)), int(np.mean(yUpper))]
            ] 
        )
        for i in range(len(linesMean)):
            cv2.line(img, (linesMean[i,0],linesMean[i,1]),(linesMean[i,2],linesMean[i,3]),color,thickness)

In [71]:
def processImage(image):
    maskedFrame = maskFrame(image)
    lines = cv2.HoughLinesP(
        maskedFrame, 
        3, 
        np.pi/180, 
        15, 
        np.array([]), 
        minLineLength= 150, 
        maxLineGap= 60) 
    print(lines)
    drawLines(image,lines)

In [72]:
cv2.startWindowThread()

video = cv2.VideoCapture('road.mp4')

while True:
    ret, frame = video.read()
    if not ret: break 
    processImage(frame)
    cv2.imshow('Video',frame) 
    cv2.waitKey(10)
    if(cv2.waitKey(1)& 0XFF == ord('q')):
        break
video.release()
cv2.destroyAllWindows()
    

[[[486 310 703 446]]

 [[494 312 870 538]]

 [[248 481 467 310]]

 [[237 489 396 361]]

 [[187 512 474 311]]

 [[199 515 459 320]]

 [[702 438 869 539]]

 [[216 491 470 313]]

 [[309 430 463 314]]

 [[509 320 671 425]]]
[[[638 404 845 538]]

 [[490 310 865 536]]

 [[509 321 740 460]]]
[[[511 323 868 538]]

 [[487 311 758 481]]

 [[490 310 801 497]]]
[[[661 419 846 539]]

 [[482 310 704 438]]]
[[[660 420 849 538]]

 [[486 310 653 407]]

 [[301 428 470 310]]

 [[308 429 473 313]]

 [[305 429 471 313]]]
[[[495 312 776 481]]

 [[508 324 790 501]]

 [[569 358 870 539]]

 [[293 443 470 310]]

 [[289 443 466 310]]

 [[593 372 870 538]]]
[[[491 310 870 538]]

 [[491 313 799 506]]]
[[[494 315 851 538]]

 [[491 310 872 539]]

 [[520 328 768 477]]]
[[[495 316 702 446]]

 [[662 413 871 538]]

 [[556 350 864 535]]

 [[309 422 474 311]]

 [[310 423 471 314]]

 [[608 381 871 539]]]
[[[596 377 865 539]]

 [[591 379 841 536]]

 [[296 441 476 311]]

 [[289 439 470 312]]

 [[508 326 699 446]]

 [[482 311